In [63]:
%load_ext autoreload
%autoreload 2

import sqlite3
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from utils.dummy import dummy_data
from utils.dummy import dummy_doi
from utils.utils import get_citation_doi
from utils.utils import get_paper_citation_pairs
from utils.utils import get_cited_by_doi
from acm_api.acm import acm_meta
from evaluation.evaluate import Train_Test_Split_Plain
import matplotlib.pyplot as plt
from cf.cf import CF
from cf.cf_unit import CFU
from cf.cf_pr import CFUPR
from cf.cf_hits import CFUH

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h2>Processing the input file with doi of papers</h2>

In [64]:
doi_list = []
data = {}
data_citedby = {}

# reading the input file with doi of papers
with open('doi/doi.txt', 'r') as doi_file:
    lines = doi_file.readlines()
    
for line in lines:
    doi_list.append(line.strip())
    
con = sqlite3.connect('data.db')
cur = con.cursor()

is_table_exists = cur.execute(
    """
    SELECT name FROM sqlite_master WHERE type='table'
    AND name='PAPERS'; """).fetchall()

if is_table_exists == []:
    cur.execute(""" 
        CREATE TABLE PAPERS (doi TEXT, data TEXT);
    """)
    
print('processing doi....')
for doi in tqdm(doi_list):
    print(doi)
    cursor = con.execute("SELECT * FROM PAPERS WHERE doi=?;", (doi,))
    fetch_data = cursor.fetchone()
    
    if fetch_data is None:
        temp = acm_meta(doi)
        insert_statement = 'INSERT INTO PAPERS(doi, data) VALUES(?, ?)'
        
        con.execute(insert_statement, (doi, str(temp)))
        data[doi] = get_citation_doi(temp)
        
    else:
        data[doi] = get_citation_doi(fetch_data[1])
        data_citedby[doi] = get_cited_by_doi(fetch_data[1])
con.commit()
con.close()

processing doi....


  0%|          | 0/7 [00:00<?, ?it/s]

10.1145/3439723
extracting citation's doi....


  0%|          | 0/155 [00:00<?, ?it/s]

extracting cited_by doi....


...

  0%|          | 0/18 [00:00<?, ?it/s]

extracting cited_by doi....


  0%|          | 0/4 [00:00<?, ?it/s]

10.1145/3487891
extracting citation's doi....


  0%|          | 0/136 [00:00<?, ?it/s]

In [65]:
# Paper's doi and it's citations
# data
data_citedby

{'10.1145/3439723': ['10.1145/3472290',
  '10.1155/2021/9249387',
  '10.1155/2021/9206440',
  '10.1145/3448734.3450467',
  '10.1145/3458280'],
 '10.1145/3446374': ['10.1145/3458025'],
 '10.1145/3459992': ['10.1145/3491231',
  '10.1155/2022/9506418',
  '10.1155/2021/7557361',
  '10.1155/2021/4873574',
  '10.1155/2021/3697536',
  '10.1155/2021/4329219',
  '10.1155/2021/8944618',
  '10.1155/2021/7965093',
  '10.1155/2021/4412294',
  '10.1155/2021/8548981',
  '10.1155/2021/1836014',
  '10.1155/2021/8586016',
  '10.1155/2021/9723379',
  '10.1155/2021/3265934',
  '10.1155/2021/5340116'],
 '10.1145/3301282': ['10.1145/3494520',
  '10.1145/3439723',
  '10.1145/3463475',
  '10.1145/3448734.3450467',
  '10.1145/3459992',
  '10.1002/ett.4145',
  '10.1145/3340531.3411884',
  '10.1007/978-3-030-62460-6_43',
  '10.1145/3415958.3433054',
  '10.1155/2020/1459107'],
 '10.1145/3322645.3322656': [],
 '10.1145/3487891': []}

<h2>Code for creating the rating matrix</h2>

In [41]:
# Store the citation space
citation_set = set()

# Store the citation space including the cited_by space
citation_set_with_cited_by = set()

# Here we are capturing the all the citation for a given paper
# Later we can use this to know the citation space
for key in data.keys():
    citation_set.update(data[key])
    citation_set_with_cited_by.update(data[key)
                                           
# Here we are appeding                                           
for key in data_citedby.keys():
    citation_set_with_cited_by.update(data_citedby[key)

index_ = data.keys()

# This dataframe does not include the cited_by data
df = pd.DataFrame(np.zeros((len(index_), len(citation_set))), index=index_, columns=citation_set)

# This dataframe include the cited_by data
df = pd.DataFrame(np.zeros((len(index_), len(citation_set))), index=index_, columns=citation_set)

for i in index_:
    for j in citation_set:
        if j in data[i]:
            df.loc[i][j] = 1

df.to_csv('citation-web/matrix-way/citation-matrix.csv')

<h2>Creating the citation pair</h2>

In [42]:
citation_pair_path = get_paper_citation_pairs(df)

writing citation pairs....


  0%|          | 0/6 [00:00<?, ?it/s]

In [43]:
########################################
	# Experiment 1
	# Plain CF
	
	
	########################################


	########################################
	# Experiment 1
	# Plain CF
	
	# TO DO
	########################################

In [44]:
evaluate = Train_Test_Split_Plain(CFU, min_ref_limit=5, train_size=0.5)
evaluate.fit(data)

The following doi's are ignored because of low reference :
{'10.1145/3322645.3322656': 3}
writing citation pairs....


  0%|          | 0/5 [00:00<?, ?it/s]

removing the indexes with no citations....
normalizing dataframe....


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 2779.53it/s]


creating similarity matrix....


...

computing recommendations for : 10.1145/3439723


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 1725.53it/s]

finishing evaluation....


  0%|          | 0/3 [00:00<?, ?it/s]

0.0

In [45]:
evaluate = Train_Test_Split_Plain(CF, min_ref_limit=5, train_size=0.5)
evaluate.fit(data)

The following doi's are ignored because of low reference :
{'10.1145/3322645.3322656': 3}
writing citation pairs....


  0%|          | 0/5 [00:00<?, ?it/s]

Creating similarity matrix....


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 85/85 [00:00<00:00, 99.44it/s]


Computing recommendations for : 10.1145/3487891


...

Computing recommendations for : 10.1145/3439723


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:00<00:00, 1769.73it/s]

finishing evaluation....


  0%|          | 0/3 [00:00<?, ?it/s]

0.05441350331744319

In [9]:
evaluate = Train_Test_Split_Plain(CFUPR, min_ref_limit=5, train_size=0.5, normalize_similarity=True)
evaluate.fit(data)

The following doi's are ignored because of low reference :
{}
writing citation pairs....


  0%|          | 0/99 [00:00<?, ?it/s]

removing the indexes with no citations....
page rank normalization on dataframe....


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:00<00:00, 5083.44it/s]


creating similarity matrix....


...

Computing recommendations for : 2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 [00:00<00:00, 419.48it/s]

finishing evaluation....


  0%|          | 0/50 [00:00<?, ?it/s]

0.1125924824893503

In [10]:
evaluate = Train_Test_Split_Plain(CFUH, min_ref_limit=5, train_size=0.5, normalize_similarity=True)
evaluate.fit(data)

The following doi's are ignored because of low reference :
{}
writing citation pairs....


  0%|          | 0/99 [00:00<?, ?it/s]

removing the indexes with no citations....
page rank normalization on dataframe....


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:00<00:00, 5089.74it/s]


creating similarity matrix....


...

Computing recommendations for : 2


0it [00:00, ?it/s]

finishing evaluation....


  0%|          | 0/50 [00:00<?, ?it/s]

0.0

In [46]:
default_path='citation-web/'
file_ = 'matrix-way/citation-matrix.csv'
file_pair = 'pair-way/citation-pairs.txt'

df = pd.read_csv(default_path + file_)
index_col = df.columns[0]
df.set_index(df[index_col], inplace=True)

df.index = df.index.astype(str, copy = False)
df.drop([index_col], axis=1, inplace=True)

# clean
indexes = df.index.values.copy()
for index in indexes:
    if not (1 in df.loc[index].values):
        df.drop(index, inplace=True)

from ranking.hits import Ranking

file_name = default_path + file_pair
default_temp_file_path  = "temp/pair-way/temp-hits-base-node.txt"

root_nodes= set(df.index.values)
potential_hub_and_auth_nodes = []

In [47]:
def __potential_hub_and_auth__():
		"""
		This method is used to find all the pages that link to the 
		nodes/papers in the root nodes. These are called potential hubs
		"""
		potential_hub_and_auth_nodes = []
		with open(file_name, 'r') as f:
			for line in f:
				from_, to_ = line.strip().replace('\n', '').split(',')

				if to_ in root_nodes:
					potential_hub_and_auth_nodes.append(from_)
				elif from_ in root_nodes:
					potential_hub_and_auth_nodes.append(to_)

		return potential_hub_and_auth_nodes 
    

In [48]:
potential_hub_auth_nodes = set(__potential_hub_and_auth__()) - root_nodes

In [49]:
def __base__():
		"""
		This method is used to create a base nodes for
		a given root nodes

		What is root nodes?
		These are the most relevant nodes for a given user
		query

		In our case, these are the query list of papers

		To form the base nodes, we find all the nodes or
		papers that has out-going edges to the root nodes
		and relation between them.
		"""
		required_nodes = set(root_nodes)
		required_nodes.update(potential_hub_auth_nodes)

		base_network = []

		with open(file_name) as f:
			for line in f:
				from_, to_ = line.strip().replace('\n', '').split(',')

				if (from_ in required_nodes) and (to_ in required_nodes):
					base_network.append(line)

		with open(default_temp_file_path, 'w') as f:
			for edge in base_network:
				f.write(edge)
                
__base__()

In [50]:
class Node:
	"""
	This class is used to create a dictionary of nodes
	Each node class has the following data
	a) vertex
	b) meta
	c) out_link
	d) in_link
	"""
	def __init__(self):
		self.vertex = None
		self.meta = None
		self.score = 1.0
		self.out_link = set()
		self.in_link = set()

def __create__():
		""" 
		This method is used to create a graph and
		return the dict with key as node name and
		value as the Node class
		"""
		node_dict = {}
		with open(default_temp_file_path) as f:
			for line in f:
				from_, to_ = line.replace('\n', '').strip().split(',')

				if not node_dict.get(from_, False):
					temp = Node()
					temp.vertex = from_
					node_dict[from_] = temp


				if not node_dict.get(to_, False):
					temp = Node()
					temp.vertex = to_
					node_dict[to_] = temp


				from_node = node_dict[from_]
				to_node = node_dict[to_]

				from_node.out_link.add(to_)
				to_node.in_link.add(from_)

		return node_dict
    




In [51]:
data = __create__()

In [53]:
def __assign_auth_score__():
		"""
		This method is used to assign the authorative score to
		all the nodes in the base network. Default we are assigning
		value 1 to all the nodes in the base network
		"""
		temp_dict = {}
		for key in data.keys():
			temp_dict[key] = 1.0 

		return temp_dict
    


auth_score = __assign_auth_score__()
hub_score = __assign_auth_score__()

In [54]:
auth_score

{'10.1145/3439723': 1.0,
 '10.5555/3294996.3295059': 1.0,
 '10.1109/AVSS.2009.58': 1.0,
 '10.1109/CVPR.2014.487': 1.0,
 '10.1145/3072959.3073659': 1.0,
 '10.1145/2601097.2601122': 1.0,
 '10.1023/A%3A1026543900054': 1.0,
 '10.1145/3301282': 1.0,
 '10.1007/BF01584320': 1.0,
 '10.1137/S0363012995293694': 1.0,
 '10.1145/1531326.1531330': 1.0,
 '10.5555/3295222.3295327': 1.0,
 '10.1016/j.patcog.2012.06.021': 1.0,
 '10.5555/2188385.2188410': 1.0,
 '10.1145/3077136.3080842': 1.0,
 '10.1609/aaai.v33i01.33018901': 1.0,
 '10.1145/3446374': 1.0,
 '10.1162/neco.2006.18.7.1527': 1.0,
 '10.1006/jcss.1997.1504': 1.0,
 '10.1007/s11263-019-01265-2': 1.0,
 '10.1109/access.2019.2905015': 1.0,
 '10.1162/089976698300017746': 1.0,
 '10.1145/3209978.3210184': 1.0,
 '10.1145/3459992': 1.0,
 '10.1109/ALLERTON.2019.8919758': 1.0,
 '10.1145/3340531.3411970': 1.0,
 '10.1109/TPAMI.2010.77': 1.0,
 '10.1145/3052973.3053009': 1.0,
 '10.1007/11681878_14': 1.0,
 '10.1145/3203217.3203241': 1.0,
 '10.1109/SECURWARE.2009.

In [55]:
iterations = 500

In [56]:
def __calculate__():
    global auth_score, hub_score
    for _ in range(iterations):
        temp_auth = {}
        temp_hub = {}
        total_auth = 0.0
        total_hub = 0.0
        for key in data.keys():
            temp_val_auth = sum([hub_score[i] for i in data[key].in_link])
            temp_val_hub = sum([auth_score[i] for i in data[key].out_link])
            temp_auth[key] = temp_val_auth
            temp_hub[key] = temp_val_hub
            total_auth += temp_val_auth
            total_hub += temp_val_hub
        for key in data.keys():
            if total_auth > 0:
                temp_auth[key] = temp_auth[key]/total_auth
            else: 
                temp_auth[key] = 0.0
            
            if total_hub > 0:
                temp_hub[key] = temp_hub[key]/total_hub
            else:
                temp_hub[key] = 0.0
        auth_score = temp_auth.copy()
        hub_score = temp_hub.copy()

In [57]:
def scores():
		scores = {}
		__calculate__()

		for key in data.keys():
			scores[key] = auth_score[key]

		return scores
    
scores()

{'10.1145/3439723': 0.0,
 '10.5555/3294996.3295059': 0.01453616891257324,
 '10.1109/AVSS.2009.58': 0.002824850481569262,
 '10.1109/CVPR.2014.487': 0.002824850481569262,
 '10.1145/3072959.3073659': 0.002824850481569262,
 '10.1145/2601097.2601122': 0.002824850481569262,
 '10.1023/A%3A1026543900054': 0.00291437451523383,
 '10.1145/3301282': 0.017166402223335965,
 '10.1007/BF01584320': 0.002824850481569262,
 '10.1137/S0363012995293694': 0.002824850481569262,
 '10.1145/1531326.1531330': 0.002824850481569262,
 '10.5555/3295222.3295327': 0.028877720654339942,
 '10.1016/j.patcog.2012.06.021': 0.002824850481569262,
 '10.5555/2188385.2188410': 0.002824850481569262,
 '10.1145/3077136.3080842': 0.017166402223335965,
 '10.1609/aaai.v33i01.33018901': 0.002824850481569262,
 '10.1145/3446374': 0.0,
 '10.1162/neco.2006.18.7.1527': 8.95240336645674e-05,
 '10.1006/jcss.1997.1504': 8.95240336645674e-05,
 '10.1007/s11263-019-01265-2': 8.95240336645674e-05,
 '10.1109/access.2019.2905015': 8.95240336645674e-

In [58]:
def scores():
		scores = {}
		__calculate__()

		for key in data.keys():
			scores[key] = auth_score[key]

		return scores
    
scores()

{'10.1145/3439723': 0.0,
 '10.5555/3294996.3295059': 0.01453616891257324,
 '10.1109/AVSS.2009.58': 0.002824850481569262,
 '10.1109/CVPR.2014.487': 0.002824850481569262,
 '10.1145/3072959.3073659': 0.002824850481569262,
 '10.1145/2601097.2601122': 0.002824850481569262,
 '10.1023/A%3A1026543900054': 0.00291437451523383,
 '10.1145/3301282': 0.017166402223335965,
 '10.1007/BF01584320': 0.002824850481569262,
 '10.1137/S0363012995293694': 0.002824850481569262,
 '10.1145/1531326.1531330': 0.002824850481569262,
 '10.5555/3295222.3295327': 0.028877720654339942,
 '10.1016/j.patcog.2012.06.021': 0.002824850481569262,
 '10.5555/2188385.2188410': 0.002824850481569262,
 '10.1145/3077136.3080842': 0.017166402223335965,
 '10.1609/aaai.v33i01.33018901': 0.002824850481569262,
 '10.1145/3446374': 0.0,
 '10.1162/neco.2006.18.7.1527': 8.95240336645674e-05,
 '10.1006/jcss.1997.1504': 8.95240336645674e-05,
 '10.1007/s11263-019-01265-2': 8.95240336645674e-05,
 '10.1109/access.2019.2905015': 8.95240336645674e-